#Nilanjan Debnath MDS201919

In [ ]:
!pip install unidecode

     |████████████████████████████████| 245kB 24.6MB/s 


Importing libraries for the pytorch implementation

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os

from tqdm import tqdm
import unidecode
import string
import random
import time
import math


In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
def generate(decoder, prime_str='A', predict_len=100, temperature=0.4, isCuda=False):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    if isCuda:
        hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    l = 1
    p = 0
    while(p<predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        if isCuda:
            inp = inp.cuda()
        if(l != len(predicted.split(' '))): 
          l += 1
          p = p+1
    while(True):
      if(predicted[-1] == '\n' or predicted[-1] == '.'): break
      output, hidden = decoder(inp, hidden)
        
      # Sample from the network as a multinomial distribution
      output_dist = output.data.view(-1).div(temperature).exp()
      top_i = torch.multinomial(output_dist, 1)[0]

      # Add predicted character to string and use as next input
      predicted_char = all_characters[top_i]
      predicted += predicted_char
      inp = Variable(char_tensor(predicted_char).unsqueeze(0))
      if isCuda:
        inp = inp.cuda()
      #print(predicted)


    return predicted

# Run as standalone script
#if __name__ == '__main__':

# Parse command line arguments
#    argparser = argparse.ArgumentParser()
#    argparser.add_argument('filename', type=str)
#    argparser.add_argument('-p', '--prime_str', type=str, default='A')
#    argparser.add_argument('-l', '--predict_len', type=int, default=100)
#    argparser.add_argument('-t', '--temperature', type=float, default=0.8)
#    argparser.add_argument('--cuda', action='store_true')
#    args = argparser.parse_args()

#    decoder = torch.load(args.filename)
#    del args.filename
#    print(generate(decoder, **vars(args)))


In [ ]:
all_characters = string.printable
n_characters = len(all_characters)

def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

# Turning a string into a tensor

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


In [ ]:
model = 'gru'
n_epochs = 10000
print_every = 100
hidden_size = 100
n_layers = 1
learning_rate = 0.01
chunk_len = 100
batch_size = 100
isCuda = True

The only pre processing that has been done is to remove all non ASCII characters. (Only the papers with english characters are used, other papers with non-english characters are not used).

In [ ]:
file, file_len = read_file('drive/MyDrive/data/corpus9.txt')

In [ ]:
def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    if isCuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

In [ ]:
def train(inp, target):
    hidden = decoder.init_hidden(batch_size)
    if isCuda:
        hidden = hidden.cuda()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()
    return loss.data.item() / chunk_len

In [ ]:
def save():
    save_filename = os.path.splitext(os.path.basename('drive/MyDrive/data/corpus4.txt'))[0] + '.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

In [ ]:
decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
)

In [ ]:
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
decoder.cuda()

CharRNN(
  (encoder): Embedding(100, 100)
  (rnn): GRU(100, 100)
  (decoder): Linear(in_features=100, out_features=100, bias=True)
)

In [ ]:
start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % n_epochs)
    for epoch in tqdm(range(1, n_epochs + 1)):
        loss = train(*random_training_set(chunk_len, batch_size))
        loss_avg += loss

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(generate(decoder, 'Wh', 100, isCuda = True), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()


  0%|          | 1/10000 [00:00<24:43,  6.74it/s]

Training for 10000 epochs...



  1%|          | 99/10000 [00:14<23:44,  6.95it/s]

 14s (100 1%) 1.8995]



  1%|          | 101/10000 [00:18<2:37:28,  1.05it/s]

...

Wh COVID-19 Pandemic and Infection and Infectious Disease Influenza Article for Decreased and Article Stround Article Protein in Acute an and infection
Interesid (Management for Coronavirus) (Wringing) (Writing the COVID-19) Impliation and COVID-19 Pandemic Detection of Community and SARS-CoV-2 infectious for a disease patients with infection of surgery of virus en a patients with patients with and in the the COVID-19 pandemic
A protein disease during the respiratory distribution of respiratory control of a patients with an porspective in an and infection in patients with patients with servic and respiratory in the experience of a study of patients with an epidemic SARS-CoV-2 in a study and review of a contribution of a case mics and sells of a in hedical and the enouction of the a contaning and and survey of an an effect of the COVID-19 in the a virus in Influenza An Covid-19 Patients with SARS-CoV-2 and a COVID-19 and and respiratory protein in the COVID-19 pandemic management in an 


100%|█████████▉| 9999/10000 [27:41<00:00,  5.94it/s]

 41s (10000 100%) 1.6036]



100%|██████████| 10000/10000 [27:42<00:00,  6.01it/s]

Whances of Survey Acute Structure of Species of Model Respiratory Serience of Syndrome Suppreadiation for SARS-CoV-2 model
Comparing and infection?
Journal Pre-proof Epidemic and and the stress of the a postalitis of the and infection and expression of the neutralization of the COVID-19 pandemic of the Term Socieation of Supplyoty of Control Survey of SARS-CoV-2 a proventifications for the infection of the COVID-19 pandemic in the the COVID-19 pandemic treatment of the diagnosis of the COVID-19 pandemic in an influenza
Score Support Management of Comple in Infections of Species for Hospitalized Systems Health Sete Sequences and Community of Self-COVID-19 endociling in the COVID-19 patients with COVID-19 pandemic in the complex and comparismption of respiratory comparition of the COVID-19 pandemic and of the and hemange severe using the COVID-19 Pandemic and Syndrome to Mediated Protein Disease for the and and Sethory of Surgery System Model Strease Protein Socieation of SARS-CoV-2 pati

predict_len decides at most 200 word sentences

In [ ]:
gen = generate(decoder,'C',predict_len = 200, temperature = 0.5, isCuda = True)

The output is atleast a 200 word sentence with the last sentence completed.

In [ ]:
gen

'COVID-19 in the case of the the risk of the an management on preptity of the in the concer for matic protein and and a learning and influenza viral and in the survey of health study antiviral infectious in the in patients in an perspective study of the COVID-19 pandemics\nA supprent in the SARS-CoV-2 in the respiratory review the influsing the Analysis of Wast patients in the effect of the comparide systematic struclitis\nArt respiratory severe of COVID-19 pandemic in the and study and infection\nJournal Pre-proof Infection of extrin and proces of the a protein and mediciles of the COVID-19 outbreak in Clinical A Markes of Sociessure Analysis\nPrepting to for the and strains of the and transmission in the study based to patient for presease componotest systematic interfer to health endocilative receptors for a models for responme interation of the respiratory viral impact of the COVID-19 infection of health microspective recophytention of the coatorocal of charation of the respiratory